In [2]:
import sys
sys.path.append('..')
from theano import function, config, shared, sandbox
import theano.tensor as T
import time
import theano_lstm
import numpy as np

In [26]:
reload(theano_lstm)
config.mode="FAST_COMPILE"
network_opt = theano_lstm.lstm_rnn(300,
                  [(51, 63)],
                  16, log_dir='test_log', dropout=0.2)
# network_opt.initialize_training_adadelta()

In [35]:
unit = 10
network_opt.initialize_network_weights()
old_norm = np.linalg.norm(network_opt.LSTM_stack.list_params()[0].get_value()[unit,:])
network_opt.do_max_norm_reg(max_norm=0.3)
new_norm = np.linalg.norm(network_opt.LSTM_stack.list_params()[0].get_value()[unit,:])
print "Old: {} | New: {}".format(old_norm, new_norm)

Old: 0.579810872449 | New: 0.3


In [3]:
num_examples = 9
input_seq = np.random.normal(size=(2200, 300, num_examples)).astype('float32')
seq_lengths = [np.random.randint(low=500,high=2200) for x in range(num_examples)]
answers = np.zeros(shape=(16, num_examples)).astype('float32')
answers[5, :] = 1

In [5]:
%%time
network_opt.adadelta_step_train(input_seq, seq_lengths, answers)

CPU times: user 7.34 s, sys: 100 ms, total: 7.44 s
Wall time: 7.44 s


array(29.527252197265625, dtype=float32)

In [5]:
network_opt.LSTM_stack.list_masks()[-2].get_value().shape

(1, 51)

In [6]:
for i in range(len(network_opt.LSTM_stack.list_params())):
    mask_shape = network_opt.LSTM_stack.list_masks()[i].get_value().shape
    param_shape = network_opt.LSTM_stack.list_params()[i].get_value().shape
    print "{} || {}".format(mask_shape, param_shape)

(51, 1) || (51, 465)
(51, 1) || (51, 1)
(51, 1) || (51, 402)
(51, 1) || (51, 1)
(51, 1) || (51, 414)
(51, 1) || (51, 1)
(51, 1) || (51, 465)
(51, 1) || (51, 1)
(1, 51) || (63, 51)
(63, 1) || (63, 1)


In [8]:
network_opt.LSTM_stack.list_params()[-2].get_value().shape

(63, 51)